In [ ]:
## set covid19_datasciente as path for python find bulletin package
import sys
from pathlib import Path
from os import getcwd
from os.path import join, basename

sys.path.append(str(Path(getcwd()).parent.parent))

In [ ]:
import numpy as np
import pandas as pd
import re
from epiweeks import Week
from unidecode import unidecode
from datetime import date, timedelta, datetime
from os.path import dirname, join, isfile, isdir
from bulletin import default_input, default_output

path_file = join('C:\\', 'SESA', 'covid19datascience-dev', 'bulletin', 'database', 'gal')
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 50)

In [ ]:
def normalize_rs(text):
    parts = filter(lambda x: len(x) > 0,text.split("ª"))
    parts = filter(lambda x: '-' not in x, parts)
    return " ".join(parts)

In [ ]:
# gal_2020 = pd.read_pickle(join(path_file, 'banco_gal_2020-bz2.pkl'), compression='bz2')
# gal_2020 = gal_2020[['id', 'data_requisicao', 'unidade_requisitante_municipio', 'unidade_requisitante_uf', 'ibge', 'municipio', 'regional', 'exame', 'metodo', 'resultado', 'virus', 'data_liberacao', 'laboratorio_executor', 'status', 'classificacao']]
# gal_2020[['id', 'data_requisicao']].groupby('data_requisicao').count().sort_values('data_requisicao')

In [ ]:
# gal_2021 = pd.read_pickle(join(path_file, 'banco_gal_2021-bz2.pkl'), compression='bz2')
# gal_2021 = gal_2021[['id', 'data_requisicao', 'unidade_requisitante_municipio', 'unidade_requisitante_uf', 'ibge', 'municipio', 'regional', 'exame', 'metodo', 'resultado', 'virus', 'data_liberacao', 'laboratorio_executor', 'status', 'classificacao']]
# # gal_2020[['id', 'data_requisicao']].groupby('data_requisicao').count().sort_values('data_requisicao')

In [ ]:
gal_2021_complemento = pd.read_csv(join(path_file, 'consulta-gal-complemento.csv'), sep = ';', usecols=['id', 'data_requisicao', 'unidade_requisitante_municipio', 'unidade_requisitante_uf', 'ibge', 'municipio', 'regional', 'exame', 'metodo', 'resultado', 'virus', 'data_liberacao', 'laboratorio_executor', 'status', 'classificacao'])
gal_2021_complemento[['id', 'data_requisicao']].groupby('data_requisicao').count().sort_values('data_requisicao')

In [ ]:
# gal = pd.concat([gal_2020, gal_2021])
gal = pd.concat([gal_2021_complemento])
# gal[['id', 'data_requisicao']].groupby('data_requisicao').count().sort_values('data_requisicao')

In [ ]:
boletim = gal.loc[gal["virus"] == 'Coronavírus SARS-CoV2'].copy()
# boletim = boletim.loc[boletim['unidade_requisitante_uf'] == 'PARANA']

In [ ]:
boletim['data_liberacao'] = pd.to_datetime(boletim['data_liberacao'], format = '%Y-%m-%d', errors= 'coerce')

In [ ]:
boletim = boletim.loc[boletim['regional']!='Outra UF']
boletim['regional'] = boletim['regional'].apply(lambda row: int(normalize_rs(row)))
boletim = boletim.loc[boletim['data_liberacao'] >= pd.to_datetime(Week.thisweek().startdate() - timedelta(7))]
boletim = boletim.loc[boletim['data_liberacao'] <= pd.to_datetime(Week.thisweek().enddate() - timedelta(7))]

In [ ]:
descartado = boletim.loc[boletim['classificacao'] == 'Descartado']
descartado = descartado[['id', 'regional']].groupby(['regional']).count().reset_index()
descartado = descartado.rename(columns={'id':'Descartado'})

In [ ]:
confirmado = boletim.loc[boletim['classificacao'] == 'Confirmado']
confirmado = confirmado[['id', 'regional']].groupby(['regional']).count().reset_index()
confirmado = confirmado.rename(columns={'id':'Confirmado'})

In [ ]:
testes = pd.DataFrame([x for x in range(1,23)],columns=['regional'])
testes = testes.merge(on = 'regional',how = 'left', right = confirmado).sort_values('regional')
testes = testes.merge(on = 'regional',how = 'left', right = descartado).sort_values('regional')
testes = testes.fillna(0).astype(int)
testes

In [ ]:
testes.to_excel(join(default_output, 'matriz','TABELA_testes_gal_por_rs.xlsx'), index = False)